In [1]:
# Fix path to be able to import classes
import sys
from pathlib import Path

# Add the src folder to the Python path
src_path = Path("../src").resolve()  # Adjust the relative path based on where your notebook is
sys.path.append(str(src_path))

<b><u>Exercise 3.1</u></b>: Comparing SelfAttention_v1 and SelfAttention_v2.

nn.Linear in SelfAttention_v2 uses a different weight initialisation scheme as nn.Parameter(torch.rand(d_in, d_out)) used in SelfAttention_v1, which causes the mechanisms to produce different results. To check that both implementations are otherwise similar, we can transfer the weight matrices from object v2 to v1, such that both objects then produce the same results. 

Correctly assign the weights from an instance of SelfAttention_v2 to SelfAttention_v1.

In [2]:
import torch
from Chapter03 import SelfAttention_v1, SelfAttention_v2, inputs, d_in, d_out

torch.manual_seed(42)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[1.3751, 0.8610],
        [1.4201, 0.8892],
        [1.4198, 0.8890],
        [1.3533, 0.8476],
        [1.3746, 0.8606],
        [1.3620, 0.8532]], grad_fn=<MmBackward0>)


In [3]:
torch.manual_seed(42)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[0.3755, 0.2777],
        [0.3761, 0.2831],
        [0.3761, 0.2833],
        [0.3768, 0.2763],
        [0.3754, 0.2836],
        [0.3772, 0.2746]], grad_fn=<MmBackward0>)


In [9]:
print("V1 class weight:", sa_v1.W_key)
print("\nV2 class weight:", sa_v2.W_key.weight)

V1 class weight: Parameter containing:
tensor([[0.2566, 0.7936],
        [0.9408, 0.1332],
        [0.9346, 0.5936]], requires_grad=True)

V2 class weight: Parameter containing:
tensor([[-0.2811,  0.3391,  0.5090],
        [-0.4236,  0.5018,  0.1081]], requires_grad=True)


In [11]:
print("V2 class weight transposed:\n", sa_v2.W_key.weight.T)

V2 class weight transposed:
 tensor([[-0.2811, -0.4236],
        [ 0.3391,  0.5018],
        [ 0.5090,  0.1081]], grad_fn=<PermuteBackward0>)


In [14]:
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)

In [15]:
torch.manual_seed(42)
print(sa_v1(inputs))

tensor([[0.3755, 0.2777],
        [0.3761, 0.2831],
        [0.3761, 0.2833],
        [0.3768, 0.2763],
        [0.3754, 0.2836],
        [0.3772, 0.2746]], grad_fn=<MmBackward0>)


In [17]:
equal = torch.equal(sa_v1(inputs), sa_v2(inputs))
print("Tensors are equal:", equal) 

Tensors are equal: True
